In [1]:
import ee
import datetime
import os
import itertools
import sys
import collections

from pprint import pprint
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

import geemap

import subprocess
from subprocess import PIPE

In [2]:
ee.Initialize()

# append the parent path into sys-path so wen can import necessary modules
sys.path.append('../../')

In [3]:
from BackGround_modules.Class_1_Make_fourier_imgs import Make_Fourier

#### Step_0_Define_Basic_Parameters

In [4]:
# Define the time-range
# just get one_range from every different sensors
year_range = [(f'{i}-01-01',f'{i+2}-12-31') for i in range(1990,2020,3)][:2]

In [5]:
year_range

[('1990-01-01', '1992-12-31'), ('1993-01-01', '1995-12-31')]

##### Adjust below code to your research area

In [6]:
# imprt North_China_Plain boundary
Target_area = ee.FeatureCollection("users/wangjinzhulala/China_built_up/01_Boundary_shp/China_zone")\
                .filterMetadata('NAME1','equals','西北')

In [7]:
path = 'users/wensomone666/China_built_up/01_Data_preparation/01_Fourier_img'

#### Step_1_Loop through each year_range to create the harmonic imgs

In [8]:
Fourier_imgs = {}
Amplitude_Phase_imgs = {}

for start,end in year_range:
    
    # define the year_name
    year_name = f'{start[:4]}_{end[:4]}'
    
    # instatiate the instance
    Fourier_instance = Make_Fourier(start_date=start,
                                    end_date=end,
                                    area=Target_area)
    
    # make result imgs
    Fourier_instance.Stp_1_Create_hamonic_names()
    Fourier_instance.Stp_2_Add_harmonics()
    Fourier_instance.Stp_3_Harmonic_fit()
    
    # Fourier img
    Fourier_imgs[year_name] = Fourier_instance.harmonicTrendCoefficients
    
    # amplitude_phase img
    Amplitude_Phase_imgs[year_name] = Fourier_instance.Amplitude_Phase_img

Analyzing the images of 1990_1992
Analyzing the images of 1993_1995


##### Step_2_Export the imgs

In [9]:
for year,img in Fourier_imgs.items():

    # construct the export name    
    export_name = f'Fourier_{year}'

        
    # construct the export task
    task = ee.batch.Export.image.toAsset(image= img,
                                         description=export_name,
                                         assetId=f'{path}/{export_name}',
                                         region=Target_area.geometry().bounds(),
                                         scale=30,
                                         maxPixels=int(1e13))

    task.start()

    # print out the process
    print(export_name)

Fourier_1990_1992
Fourier_1993_1995


##### Add one img to the map

In [10]:
# the img
one_img = img

# the visulisation parameters
Fourier_VIS = {"bands":["NDBI_cos_1","EVI_sin_2","NDVI_sin_2"],"min":-106,"max":158}

In [12]:
# show the map for validation
Map = geemap.Map()
Map.add_basemap('HYBRID')
Map.setCenter(120.3482, 33.0212,8)

# add the img
Map.addLayer(one_img,Fourier_VIS,'Fourier_VIS')


# show the img
Map